In [187]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as re

import datetime

In [103]:
# Latitud Y Longitud de Provincias

In [188]:
lat_long = 'https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales'
html = re.get(lat_long).text
soup = bs(html, 'html.parser')

tabla = [e for e in soup.find_all('tr')]
data = [e for elem in tabla for e in elem.find_all('td')][1:]

data_map = pd.DataFrame()

data_map['Provincia'] = [data[e].text for e in range(1, len(data),8)]
data_map['Poblacion'] = [data[e].text for e in range(2, len(data),8)]
data_map['Lat'] = [data[e].text for e in range(3, len(data),8)]
data_map['Lon'] = [data[e].text for e in range(4, len(data),8)]

# A continuación, corrijo las provincias que no vienen igual escritas en la tabla de mapeo por long y latitud que en la iso.

a_corregir = [e for e in list(data_map['Provincia']) if e not in list(iso_mapear['provincia'])]
correctas = [e for e in iso_mapear['provincia'] if e not in list(data_map['Provincia'])]

data_map['Provincia'] = ['A Coruña', 'Albacete', 'Alacant', 'Almería', 'Ávila', 'Badajoz', 'Barcelona', 'Bizkaia', 'Burgos',
                         'Cáceres', 'Cádiz', 'Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba', 'Cuenca', 'Girona', 'Granada',
                         'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'Las Palmas', 'León', 'Lleida', 'La Rioja', 'Lugo', 
                         'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Ourense', 'Asturias', 'Palencia', 'Illes Balears', 'Nafarroa',
                         'Pontevedra', 'Salamanca', 'Gipuzkoa', 'Sta. Cruz de Tenerife', 'Cantabria', 'Segovia', 'Sevilla', 'Soria',
                         'Tarragona', 'Teruel', 'Toledo', 'Valéncia', 'Valladolid', 'Álava', 'Zamora', 'Zaragoza']

data_map.head()

,Provincia,Poblacion,Lat,Lon
0,A Coruña,246.047,"43,37012643","-8,39114853"
1,Albacete,170.475,"38,99588053","-1,85574745"
2,Alacant,334.418,"38,34548705","-0,4831832"
3,Almería,190.013,"36,83892362","-2,46413188"
4,Ávila,58.245,"40,65586958","-4,69771277"


In [104]:
# ISO para relacionar Codigos de Provincias y Nombres

In [189]:
iso = pd.read_csv("https://gist.githubusercontent.com/JustDevZero/139ad12e822c79496769/raw/f5a27fae77fc39bca98500b3e36130810e0020f5/provinces_es.csv")
iso.columns = [e.replace(" ", "_").lower() for e in iso.columns]

iso_mapear = pd.DataFrame()

iso_mapear['code'] = [e for e in iso['code']]
iso_mapear['provincia'] = iso['name']


dict_iso = {k : v for k, v in zip(iso_mapear['code'], iso_mapear['provincia'])}


In [190]:
pd.options.display.max_rows = 10
df = pd.read_csv("Covid_Data.csv")

df['fecha'] = pd.to_datetime(df['fecha'])
df['fecha_y_m'] = df['fecha'].dt.to_period('M')

df_provincias = df.groupby(['provincia_iso', 'fecha_y_m'], as_index = False).sum()
df_spain = df.groupby(['fecha_y_m'], as_index = False).sum()

df_provincias['provincia_iso'] = df_provincias['provincia_iso'].map(dict_iso)

df_provincias = df_provincias.merge(data_map, how = "inner", left_on = 'provincia_iso', right_on = 'Provincia')
df_provincias = df_provincias[['Provincia', 'Poblacion', 'Lat', 'Lon', 'fecha_y_m', 'num_casos', 'num_hosp', 'num_uci', 'num_def']]

df_provincias.columns = ['Provincia', 'Population', 'Latitude', 'Longitude', 'Year_Month', 'N_Cases', 'N_Hosp', 'N_UCI', 'N_Death']

df_provincias['Longitude'] = [float(e.replace(',', '.')) for e in df_provincias['Longitude']]
df_provincias['Latitude'] = [float(e.replace(',', '.')) for e in df_provincias['Latitude']]


In [186]:
df_provincias.to_excel('covid_provincias.xlsx', index = False)



In [217]:
df_provincias

,Provincia,Population,Latitude,Longitude,Year_Month,N_Cases,N_Hosp,N_UCI,N_Death
0,Alacant,334.418,38.345487,-0.483183,2020-01,0,13,0,0
1,Alacant,334.418,38.345487,-0.483183,2020-02,0,13,3,0
2,Alacant,334.418,38.345487,-0.483183,2020-03,2511,1559,232,198
3,Alacant,334.418,38.345487,-0.483183,2020-04,1324,506,61,255
4,Alacant,334.418,38.345487,-0.483183,2020-05,260,74,9,43
...,...,...,...,...,...,...,...,...,...
1372,Zamora,65.998,41.499140,-5.754948,2021-11,445,35,4,2
1373,Zamora,65.998,41.499140,-5.754948,2021-12,5159,96,5,7
1374,Zamora,65.998,41.499140,-5.754948,2022-01,12712,201,11,39
1375,Zamora,65.998,41.499140,-5.754948,2022-02,3251,115,8,35


In [300]:

provincias_map

,provincia,n_cases,n_hosp,n_uci,n_death,longitude,latitude
0,A Coruña,240844,9651,1428,1378,-8.391149,43.370126
1,Alacant,467693,18906,1769,3464,-0.483183,38.345487
2,Albacete,88472,5210,640,984,-1.855747,38.995881
3,Almería,136440,5556,860,1149,-2.464132,36.838924
4,Asturias,206534,12391,1024,2603,-5.843722,43.362322
...,...,...,...,...,...,...,...
46,Valéncia,728401,28651,2316,4614,-0.375657,39.475344
47,Zamora,40907,2545,185,660,-5.754948,41.499140
48,Zaragoza,296628,14663,1288,3363,-0.879287,41.656457
49,Álava,94121,3610,325,904,-2.672757,42.850588


In [350]:
import pydeck as pdk

provincias_map = df_provincias.groupby('Provincia', as_index = False).sum().drop(['Longitude', 'Latitude'], axis = 1)
provincias_map = pd.merge(provincias_map, df_provincias[['Provincia', 'Longitude', 'Latitude']].drop_duplicates(), on = 'Provincia', how = 'left')
provincias_map.columns = [e.lower() for e in provincias_map.columns]


r = pdk.Deck(map_style='light',
        initial_view_state=pdk.ViewState(
        latitude=provincias_map['latitude'].mean(),
        longitude=provincias_map['longitude'].mean(),
        zoom=5,
        pitch=45),
        layers=[
        pdk.Layer(
           'HexagonLayer',
           data=provincias_map,
           get_position=['longitude', 'latitude'],
           radius=18000,
           elevation_scale=1000,
           elevation_range=[0, 1000],
           get_elevation_weight = "n_cases",
           pickable=True,
           extruded=True,
           auto_highlight=True,
        )])
r

{
  "initialViewState": {
    "latitude": 39.85315581131372,
    "longitude": -3.8938955577843135,
    "pitch": 45,
    "zoom": 5
  },
  "layers": [
    {
      "@@type": "HexagonLayer",
      "autoHighlight": true,
      "data": [
        {
          "latitude": 43.37012643,
          "longitude": -8.39114853,
          "n_cases": 240844,
          "n_death": 1378,
          "n_hosp": 9651,
          "n_uci": 1428,
          "provincia": "A Coru\u00f1a"
        },
        {
          "latitude": 38.34548705,
          "longitude": -0.4831832,
          "n_cases": 467693,
          "n_death": 3464,
          "n_hosp": 18906,
          "n_uci": 1769,
          "provincia": "Alacant"
        },
        {
          "latitude": 38.99588053,
          "longitude": -1.85574745,
          "n_cases": 88472,
          "n_death": 984,
          "n_hosp": 5210,
          "n_uci": 640,
          "provincia": "Albacete"
        },
        {
          "latitude": 36.83892362,
          "longitude": -2.46413188,
          "n_cases": 136440,
          "n_death": 1149,
          "n_hosp": 5556,
          "n_uci": 860,
          "provincia": "Almer\u00eda"
        },
        {
          "latitude": 43.36232165,
          "longitude": -5.84372206,
          "n_cases": 206534,
          "n_death": 2603,
          "n_hosp": 12391,
          "n_uci": 1024,
          "provincia": "Asturias"
        },
        {
          "latitude": 38.87874339,
          "longitude": -6.97099704,
          "n_cases": 164384,
          "n_death": 1189,
          "n_hosp": 5074,
          "n_uci": 420,
          "provincia": "Badajoz"
        },
        {
          "latitude": 41.38424664,
          "longitude": 2.17634927,
          "n_cases": 1755643,
          "n_death": 14567,
          "n_hosp": 46680,
          "n_uci": 3974,
          "provincia": "Barcelona"
        },
        {
          "latitude": 43.25721957,
          "longitude": -2.92390606,
          "n_cases": 335102,
          "n_death": 3055,
          "n_hosp": 14645,
          "n_uci": 1292,
          "provincia": "Bizkaia"
        },
        {
          "latitude": 42.34113004,
          "longitude": -3.70419805,
          "n_cases": 106306,
          "n_death": 1090,
          "n_hosp": 5741,
          "n_uci": 475,
          "provincia": "Burgos"
        },
        {
          "latitude": 43.46297885,
          "longitude": -3.80474784,
          "n_cases": 130263,
          "n_death": 789,
          "n_hosp": 6165,
          "n_uci": 799,
          "provincia": "Cantabria"
        },
        {
          "latitude": 39.98640809,
          "longitude": -0.03688142,
          "n_cases": 160344,
          "n_death": 1053,
          "n_hosp": 5835,
          "n_uci": 707,
          "provincia": "Castell\u00f3"
        },
        {
          "latitude": 35.88810209,
          "longitude": -5.30675127,
          "n_cases": 19744,
          "n_death": 146,
          "n_hosp": 562,
          "n_uci": 72,
          "provincia": "Ceuta"
        },
        {
          "latitude": 38.98651781,
          "longitude": -3.93131981,
          "n_cases": 115734,
          "n_death": 1900,
          "n_hosp": 7877,
          "n_uci": 481,
          "provincia": "Ciudad Real"
        },
        {
          "latitude": 40.07653762,
          "longitude": -2.13152306,
          "n_cases": 51871,
          "n_death": 713,
          "n_hosp": 4048,
          "n_uci": 238,
          "provincia": "Cuenca"
        },
        {
          "latitude": 39.47316762,
          "longitude": -6.37121092,
          "n_cases": 87176,
          "n_death": 1071,
          "n_hosp": 3427,
          "n_uci": 293,
          "provincia": "C\u00e1ceres"
        },
        {
          "latitude": 36.52171152,
          "longitude": -6.28414575,
          "n_cases": 190480,
          "n_death": 1764,
          "n_hosp": 7592,
          "n_uci": 716,
          "provincia": "C\u00e1diz"
        },
        {
          "latitude": 37.8795422